In [1]:
import pandas as pd
import numpy as np

import sqlalchemy
import urllib
import pyodbc
#import requests

## Connection Strings

In [2]:
params = urllib.parse.quote_plus('Driver={SQL Server Native Client 11.0}; SERVER=LAPTOP-B22E1F0Q\SQLEXPRESS; DATABASE=master; Trusted_Connection=yes')
engine = sqlalchemy.create_engine('mssql+pyodbc:///?odbc_connect=%s' % params, fast_executemany=True)

## Create New Database 'NewDb'

In [3]:
try:
    con = pyodbc.connect('Driver={SQL Server Native Client 11.0}; SERVER=LAPTOP-B22E1F0Q\SQLEXPRESS; DATABASE=master; Trusted_Connection=yes'
                        , autocommit=True)
 
    cursor = con.cursor()

    sql_query = 'CREATE DATABASE NewDb'
    
    cursor.execute(sql_query)

    con.commit()
    
except Exception as e:
    print(e)

finally:
    con.close()

## Upload .csv to NewDb

In [130]:
try:
    con = pyodbc.connect('Driver={SQL Server Native Client 11.0}; SERVER=LAPTOP-B22E1F0Q\SQLEXPRESS; DATABASE=master; Trusted_Connection=yes')
 
    #read file into pandas df
    users_data = pd.read_csv("C:/Users/arteu/Documents/users.csv")
    
    users_data.to_sql (
        name='users',
        con=engine,
        index=False,
        if_exists='replace'
    )
    

except Exception as e:
    print(e)

finally:
    con.close()

## Create derived daily_user_counts table

In [131]:
try:
    con = pyodbc.connect('Driver={SQL Server};'
                          'Server=LAPTOP-B22E1F0Q\SQLEXPRESS;'
                          'Database=master;'
                          'Trusted_Connection=yes;')


    sql = """

        DROP TABLE IF EXISTS daily_user_counts;
        with users as
        (
            SELECT
                year(visit_date) as Year
                , month(visit_date) as Month
                , day(visit_date) as Day
            --not sure what the difference between Count and Observed is meant to be, so assumed distinct vs. total counts even though they turned out to be the same
                , COUNT(id) as Observed
                , COUNT(DISTINCT id) as Count
            FROM master.dbo.users
            GROUP BY 
                year(visit_date)
                , month(visit_date)
                , day(visit_date)
            --order by 1,2,3,4,5
        )
        SELECT
            Year
            , Month
            , Day
            , Observed
            , Count
        INTO master.dbo.daily_user_counts --alternative way can be to create table with defined datatypes and insert that way
        From users

    """


    cursor = con.cursor()
    cursor.execute(sql)
    con.commit()

except Exception as e:
    print(e)

finally:
    con.close()

## Check results of daily_user_counts

In [132]:
try:
    con = pyodbc.connect('Driver={SQL Server};'
                          'Server=LAPTOP-B22E1F0Q\SQLEXPRESS;'
                          'Database=master;'
                          'Trusted_Connection=yes;')

    sql = 'SELECT * FROM master.dbo.daily_user_counts'
    cursor = con.cursor()
    cursor.execute(sql)



    for row in cursor:
        print(row)

except Exception as e:
    print(e)

finally:
    con.close()

(2020, 9, 7, 6, 6)
(2020, 9, 8, 2, 2)
(2020, 9, 9, 15, 15)
(2020, 9, 10, 20, 20)
(2020, 9, 11, 14, 14)
(2020, 9, 14, 10, 10)
(2020, 9, 15, 12, 12)
(2020, 9, 16, 19, 19)


## Aggregate users table in Pandas

In [133]:
#read file into pandas df
users = pd.read_csv("C:/Users/arteu/Documents/users.csv")
users

,id,first_name,last_name,age,gender,visit_date
0,16351203 2040,Leroy,Dillon,55,M,2020-09-07
1,16980104 6773,Macaulay,Griffin,14,M,2020-09-07
2,16230625 0982,Phillip,Chapman,45,M,2020-09-07
3,16180206 2123,Phillip,Moses,39,M,2020-09-07
4,16980621 8401,Tatyana,Goodman,40,F,2020-09-07
...,...,...,...,...,...,...
93,16640909 9774,Kadeem,Lindsay,65,M,2020-09-16
94,16891103 0453,Jermaine,Love,56,M,2020-09-16
95,16510720 9487,Gemma,Baird,6,F,2020-09-16
96,16430629 1396,Brent,Barr,67,M,2020-09-16


### Table: Number of Users by Day

In [134]:
users_byday = users.groupby('visit_date')['id'].count().to_frame()

#rename id count column to user_count
users_byday = users_byday.rename(columns = {'id':'user_count'})

users_byday

,user_count
visit_date,
2020-09-07,6
2020-09-08,2
2020-09-09,15
2020-09-10,20
2020-09-11,14
2020-09-14,10
2020-09-15,12
2020-09-16,19


In [135]:
#generate all date values between date min and max to explicilty fill in 0's for days with no users

dates_index = pd.date_range(users_byday.index.min(), users_byday.index.max())
users_byday.index = pd.DatetimeIndex(users_byday.index)
s = users_byday.reindex(dates_index, fill_value=0)

In [136]:
s

,user_count
2020-09-07,6
2020-09-08,2
2020-09-09,15
2020-09-10,20
2020-09-11,14
2020-09-12,0
2020-09-13,0
2020-09-14,10
2020-09-15,12
2020-09-16,19


In [137]:
#use a 7-day moving average to calculate Expected Value of subsequent day
s['MovingAverage'] = s.user_count.rolling(7).mean()

In [138]:
#Expected Value of next day's users set equal to the Moving Average of the latest date
ExpectedValue = s['MovingAverage'][s.index == s.index.max()].item()
print(ExpectedValue)

10.714285714285714


In [139]:
s

,user_count,MovingAverage
2020-09-07,6,NaN
2020-09-08,2,NaN
2020-09-09,15,NaN
2020-09-10,20,NaN
2020-09-11,14,NaN
2020-09-12,0,NaN
2020-09-13,0,8.142857
2020-09-14,10,8.714286
2020-09-15,12,10.142857
2020-09-16,19,10.714286


In [149]:
#append next date to df with Expected Value

nextdate = s.index.max() + pd.DateOffset(days=1)
row = pd.Series({'user_count':ExpectedValue,'MovingAverage':'0'},name=nextdate)
predicted_users = s.append(row)

In [150]:
predicted_users

,user_count,MovingAverage
2020-09-07,6.000000,NaN
2020-09-08,2.000000,NaN
2020-09-09,15.000000,NaN
2020-09-10,20.000000,NaN
2020-09-11,14.000000,NaN
2020-09-12,0.000000,NaN
2020-09-13,0.000000,8.14286
2020-09-14,10.000000,8.71429
2020-09-15,12.000000,10.1429
2020-09-16,19.000000,10.7143


In [151]:
try:
    predicted_users.drop('MovingAverage', axis=1, inplace=True)
except Exception as e:
    print(e)

In [152]:
predicted_users

,user_count
2020-09-07,6.000000
2020-09-08,2.000000
2020-09-09,15.000000
2020-09-10,20.000000
2020-09-11,14.000000
2020-09-12,0.000000
2020-09-13,0.000000
2020-09-14,10.000000
2020-09-15,12.000000
2020-09-16,19.000000


In [153]:
#For completeness, recalculate 7-day moving average to include latest datapoint
predicted_users['MovingAverage'] = predicted_users.user_count.rolling(7).mean()

In [154]:
predicted_users

,user_count,MovingAverage
2020-09-07,6.000000,NaN
2020-09-08,2.000000,NaN
2020-09-09,15.000000,NaN
2020-09-10,20.000000,NaN
2020-09-11,14.000000,NaN
2020-09-12,0.000000,NaN
2020-09-13,0.000000,8.142857
2020-09-14,10.000000,8.714286
2020-09-15,12.000000,10.142857
2020-09-16,19.000000,10.714286


## Upload table with projected values to SQL for appending

In [146]:
try:
    con = pyodbc.connect('Driver={SQL Server Native Client 11.0}; SERVER=LAPTOP-B22E1F0Q\SQLEXPRESS; DATABASE=master; Trusted_Connection=yes')

    
    predicted_users.to_sql (
        name='predicted_users',
        con=engine,
        index=True,
        index_label='Date',
        if_exists='replace'
    )
            

except Exception as e:
    print(e)

finally:
    con.close()

## Append data to daily_user_counts table

In [147]:
try:
    con = pyodbc.connect('Driver={SQL Server};'
                          'Server=LAPTOP-B22E1F0Q\SQLEXPRESS;'
                          'Database=master;'
                          'Trusted_Connection=yes;')


    sql = """

        --this method also appends days with 0 users that were imputed from the reindexing; can't hurt to have, easy to filter out
       
        INSERT INTO master.dbo.daily_user_counts (Year, Month, Day, Observed, Count)
        select
            YEAR(Date) as Year
            , Month(Date) as Month
            , Day(Date) as Day
            --alternative interpretation is that 0 were observed for projected values, but I kept the same methodology for consistency
            , user_count as Observed 
            , user_count as Count
        FROM [master].[dbo].[predicted_users] pu
          --Exclude appending data that is already in the table; could also do this with a join
          WHERE CONCAT(YEAR(pu.Date),'-',MONTH(pu.DATE),'-',DAY(pu.DATE)) NOT IN
          (
                SELECT CONCAT(duc.YEAR,'-',duc.MONTH,'-',duc.DAY) FROM master.dbo.daily_user_counts duc
            ) 


    """


    cursor = con.cursor()
    cursor.execute(sql)
    con.commit()

except Exception as e:
    print(e)

finally:
    con.close()